In [65]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# conda install pytorch torchvision -c pytorch 
# conda install -c anaconda joblib
import joblib
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeavePOut

# import MLPRegressor
# XGBRegressor
# LGBMRegressor
# CatBoostRegressor
# KNeighborsRegressor
# Lasso
# Ridge
# ElasticNet
# SGDRegressor

from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor


In [66]:
# data/output_df 에서 파일명에 "spring" 이 들어간 파일을 불러오는 함수 만들기
def get_spring_df(path):
    file_list = os.listdir(path)
    file_list_spring = [file for file in file_list if 'spring' in file]
    df_list = []
    for file in file_list_spring:
        df = pd.read_csv(path + '/' + file)
        df_list.append(df)
        # 지역명 컬럼 추가하여 파일명에서 추출
        df['지역명'] = file.split('_')[0]
    df = pd.concat(df_list)
    return df

# 함수 실행
spring_location_df = get_spring_df('data/output_df')

# 함수 실행
spring_weather_df = get_spring_df('data/weather')

# 파일 안에 '-' 를 0으로 변경
spring_location_df = spring_location_df.replace('-', 0)
spring_weather_df = spring_weather_df.replace('-', 0)

# spring_location_df와 spring_weather_df를 '지역명' 컬럼을 기준으로 groupby, 일시를 기준으로 정렬 merge
spring_df = pd.merge(spring_location_df, spring_weather_df, on=['지역명', '일시'], how='left').sort_values(by=['지역명', '일시']).reset_index(drop=True)
spring_df= spring_df[["일시","지역명",	"일반봄배추:면적 (ha)",	"생산량 (톤)"	,"평균기온(°C)",	"최고기온(°C)",	"최저기온(°C)",	"월합강수량(00~24h만)(mm)"	,"합계 일사량(MJ/m2)"]]
spring_df

,일시,지역명,일반봄배추:면적 (ha),생산량 (톤),평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2)
0,2000,busan,0,0,9.1,18.3,-2.5,48.1,495.90
1,2000,busan,0,0,13.2,21.4,2.8,65.4,539.21
2,2000,busan,0,0,17.3,29.1,10.7,72.0,607.84
3,2001,busan,0,0,9.6,22.7,-3.3,6.1,439.57
4,2001,busan,0,0,15.0,25.3,3.8,42.4,539.17
...,...,...,...,...,...,...,...,...,...
5813,2020,ulsan,6,281,12.3,25.5,2.9,73.2,633.67
5814,2020,ulsan,6,281,18.1,28.7,9.2,51.5,614.27
5815,2021,ulsan,10,503,10.5,21.9,-0.8,138.9,497.10
5816,2021,ulsan,10,503,13.8,25.7,3.1,73.9,608.97


In [67]:
# 머신러닝 모델에 사용할 컬럼명
cols = ['일반봄배추:면적 (ha)','생산량 (톤)', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)', '월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)']

# 머신러닝 모델 학습에 사용할 데이터셋
spring_df = spring_df[cols]

# 데이터셋을 학습용, 테스트용으로 분리 20%
train, test = train_test_split(spring_df, test_size=0.2, random_state=42)

# 학습용 데이터셋을 학습용, 검증용으로 분리 20%
train, val = train_test_split(train, test_size=0.2, random_state=42)

# 학습용 데이터셋을 X, y로 분리
X_train = train.drop(['생산량 (톤)'], axis=1)
y_train = train['생산량 (톤)']

# 검증용 데이터셋을 X, y로 분리
X_val = val.drop(['생산량 (톤)'], axis=1)
y_val = val['생산량 (톤)']

# 테스트용 데이터셋을 X, y로 분리
X_test = test.drop(['생산량 (톤)'], axis=1)
y_test = test['생산량 (톤)']

# 머신러닝 모델 학습
# MLPRegressor

mlp = MLPRegressor(random_state=42)
mlp.fit(X_train, y_train)

# MLPRegressor 모델 평가
print('MLPRegressor')
print('train score :', mlp.score(X_train, y_train))
print('val score :', mlp.score(X_val, y_val))
print('test score :', mlp.score(X_test, y_test))

# 모델 저장
joblib.dump(mlp, 'model/spring.pkl')




MLPRegressor
train score : 0.9721593953365738
val score : 0.9728682515701967
test score : 0.9708720703389555


['model/spring.pkl']

In [68]:
# model/spring.pkl 이용하여 다음연도 봄배추 생산량 예측

# 머신러닝 모델 불러오기
mlp = joblib.load('model/spring.pkl')

# 다음연도 봄배추 생산량 예측
spring_predict = mlp.predict(X_test)

# 예측값과 실제값을 비교
spring_predict = pd.DataFrame(spring_predict, columns=['예측값'])
spring_predict['실제값'] = y_test.reset_index(drop=True)
spring_predict





,예측값,실제값
0,-72.373239,0
1,-47.991849,0
2,5418.493937,5325
3,-3.561678,0
4,-350.399311,0
...,...,...
1159,-57.117059,0
1160,-102.310101,0
1161,-33.078418,0
1162,35936.495181,34482
